In [1]:
# Pandas 라이브러리 임포트
import pandas as pd
from pandas import Series, DataFrame

# 시각화를 위한 라이브러리 임포트
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.ticker as ticker
%matplotlib inline
plt.rcParams["font.family"] = "Malgun Gothic"

# 8. 상위 5개팀의 경기 최종 예측
- 이로서 독일, 프랑스, 포르투갈, 잉글랜드, 스페인 5개국의 여러 능력치와 성향을 모두 분석해보았다.
- 이후 이 5개팀들끼리의 경기, 특히 카타르 월드컵에서의 경기가 있을 때 지금까지의 분석을 바탕으로 어느 팀이 이길지 예측해보자.

In [2]:
# 상위 5개팀의 능력치 불러오기

state_data = pd.read_excel('data/5개팀종합능력치.xlsx', header=0, index_col=0)
state_data

,교체력,패스력,공격력,수비력
국가,,,,
독일,84,153,243,336
스페인,73,141,201,359
잉글랜드,85,137,205,365
포르투갈,94,146,240,301
프랑스,61,145,228,354


In [3]:
sd = state_data.reset_index()

In [30]:
fig = go.Figure(data=[
    go.Bar(name='교체력', x=sd['국가'], y=sd['교체력']),
    go.Bar(name='패스력', x=sd['국가'], y=sd['패스력']),
    go.Bar(name='공격력', x=sd['국가'], y=sd['공격력']),
    go.Bar(name='수비력', x=sd['국가'], y=sd['수비력'])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [5]:
# 상위 5개팀의 성향 불러오기

tendency_data = pd.read_excel('data/5개팀종합성향.xlsx', header=0, index_col=0)
tendency_data

,승률,득점강세시간,실점취약시간,무승부유지,승기유지,역전승,패기유지,후반승기손실,후반승기쟁취,후반추격성공,후반추격허용,패스호전성,선호패스방향,선호패스,수비전략성
국가,,,,,,,,,,,,,,,
독일,55,후반,후반,4,9,1,3,1,1,1,0,공격성,좌향,숏+백패스 위주,매우 전략적
스페인,55,중립,후반,3,7,0,1,1,4,0,4,수비성,좌향,롱패스 위주,비효율적
잉글랜드,57,후반,후반,6,10,0,1,2,3,0,1,수비성,좌향,중립,비효율적
포르투갈,55,전반,전반,2,8,1,3,2,2,1,1,중립,좌향,롱패스 위주,전략적
프랑스,39,후반,전반,3,4,0,2,1,3,4,1,공격성,좌향,숏+백패스 위주,중립


In [6]:
# 상위 5개팀의 득실점 타임라인 불러오기

timeline_data = pd.read_csv('data/5개팀득실점타임라인.csv', header=[0, 1], index_col=0)
timeline_data

득점                                               ...   실점            \
     0분대 10분대 20분대 30분대 40분대 50분대 60분대 70분대 80분대 90분대  ... 30분대 40분대 50분대   
독일     5    5    4    3    5   10    7    5    8    0  ...    0    1    1   
프랑스    1    3    2    2    3    9    4    3    3    0  ...    5    2    1   
포르투갈   2    6    1   10    3    2    5    1    7    3  ...    4    5    1   
잉글랜드   4    8    4    6    4    6    5    7    8    0  ...    1    0    3   
스페인    2    2    5    6    3    4    2    4    4    2  ...    0    1    4   

                                            
     60분대 70분대 80분대 90분대 100분대 110분대 120분대  
독일      1    5    4    1     0     0     0  
프랑스     2    0    3    1     0     0     0  
포르투갈    3    0    1    1     0     0     0  
잉글랜드    4    1    3    1     0     0     0  
스페인     3    1    2    1     0     0     0  

[5 rows x 26 columns]

## 8-1. 5개 팀의 전투력을 구한 뒤 각각 팀들의 경기 내용 예측
- 상위 5개팀의 능력치에서 전투력을 가져온 뒤, 성향에서 상성을 따질 수 있는 것을 바탕으로 가산점을 주어 경기 승패를 예측

In [7]:
# 능력치들의 합산으로 전투력 측정
state_data['전투력'] = state_data['교체력'] + state_data['패스력'] + state_data['공격력'] + state_data['수비력']

In [8]:
# 수비전략성에 따라 가산점 부여
# 전략적 = 1.05배
# 중립 = 1.0배
# 비효율적 = 0.95배

state_data.loc['독일', '전투력'] *= 1.05
state_data.loc['스페인', '전투력'] *= 0.95
state_data.loc['잉글랜드', '전투력'] *= 0.95
state_data.loc['포르투갈', '전투력'] *= 1.05
state_data.loc['프랑스', '전투력'] *= 1.0

In [9]:
power = state_data['전투력'].sort_values(ascending = False)
power = power.astype('int')
power

국가
독일      856
포르투갈    820
프랑스     788
잉글랜드    752
스페인     735
Name: 전투력, dtype: int32

In [10]:
# 나라별 공격력 시각화
# plotly 라이브러리 활용

fig = px.bar(power, x=power.index, y=power.values,
             color=power.values,
             height=500,
             title="국가별 전투력 시각화")
fig.show()

### 상위 5개팀의 전투력과 순위
- 1위 : 독일     (856점)
- 2위 : 포르투갈 (820점)
- 3위 : 프랑스   (788점)
- 4위 : 잉글랜드 (752점)
- 5위 : 스페인   (735점)
<hr>

### 프랑스 VS 잉글랜드
- 전투력을 베이스로 상성을 따져 가산점 부여

In [11]:
# 프랑스의 전투력

team1 = power.프랑스
team1

788

In [12]:
# 잉글랜드의 전투력

team2 = power.잉글랜드
team2

752

In [13]:
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['프랑스', '잉글랜드'],['득점강세시간', '실점취약시간']]
time

,득점강세시간,실점취약시간
국가,,
프랑스,후반,전반
잉글랜드,후반,후반


In [14]:
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.2)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.2)

# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  945
팀2 전투력 :  752
팀1 승리


### 프랑스 VS 잉글랜드는 프랑스의 승리 예측
<hr>

In [15]:
# 프랑스의 전투력
team1 = power.프랑스
# 독일의 전투력
team2 = power.독일
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['프랑스', '독일'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.1)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.1)
    
# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  866
팀2 전투력 :  856
팀1 승리


### 프랑스 VS 독일은 독일의 승리 예측
<hr>

In [16]:
# 프랑스의 전투력
team1 = power.프랑스
# 포르투갈의 전투력
team2 = power.포르투갈
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['프랑스', '포르투갈'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.1)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.1)
    
# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  788
팀2 전투력 :  902
팀2 승리


### 프랑스 VS 포르투갈은 포르투갈의 승리 예측
<hr>

In [17]:
# 프랑스의 전투력
team1 = power.프랑스
# 스페인의 전투력
team2 = power.스페인
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['프랑스', '스페인'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.1)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.1)
    
# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  866
팀2 전투력 :  735
팀1 승리


### 프랑스 VS 스페인은 프랑스의 승리 예측
<hr>

In [18]:
# 독일의 전투력
team1 = power.독일
# 잉글랜드의 전투력
team2 = power.잉글랜드
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['독일', '잉글랜드'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.1)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.1)
    
# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  941
팀2 전투력 :  827
팀1 승리


### 독일 VS 잉글랜드는 독일의 승리 예측
<hr>

In [19]:
# 독일의 전투력
team1 = power.독일
# 포르투갈의 전투력
team2 = power.포르투갈
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['독일', '포르투갈'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.1)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.1)
    
# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  856
팀2 전투력 :  820
팀1 승리


### 독일 VS 포르투갈은 독일의 승리 예측
<hr>

In [20]:
# 독일의 전투력
team1 = power.독일
# 스페인의 전투력
team2 = power.스페인
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['독일', '스페인'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.1)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.1)

# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  941
팀2 전투력 :  735
팀1 승리


### 독일 VS 스페인은 독일의 승리 예측
<hr>

In [21]:
# 스페인의 전투력
team1 = power.스페인
# 잉글랜드의 전투력
team2 = power.잉글랜드
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['스페인', '잉글랜드'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.2)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.2)

# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  735
팀2 전투력 :  902
팀2 승리


### 스페인 VS 잉글랜드는 잉글랜드의 승리 예측
<hr>

In [22]:
# 스페인의 전투력
team1 = power.스페인
# 포르투갈의 전투력
team2 = power.포르투갈
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['스페인', '포르투갈'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.2)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.2)

# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  735
팀2 전투력 :  820
팀2 승리


### 스페인 VS 포르투갈은 포르투갈의 승리 예측
<hr>

In [23]:
# 잉글랜드의 전투력
team1 = power.잉글랜드
# 포르투갈의 전투력
team2 = power.포르투갈
# 두 팀의 득점강세시간과 실점취약시간
time = tendency_data.loc[['잉글랜드', '포르투갈'],['득점강세시간', '실점취약시간']]
# 자신팀의 득점강세시간이 상대팀의 실점취약시간일 경우 전투력 x 1.2
if time.iloc[0, 0] == time.iloc[1, 1] :
    team1 = int(team1 * 1.2)
if time.iloc[0, 1] == time.iloc[1, 0] :
    team2 = int(team2 * 1.2)

# 승부 결과 예측
print('팀1 전투력 : ', team1)
print('팀2 전투력 : ', team2)
if team1 > team2 : print('팀1 승리')
else : print('팀2 승리')

팀1 전투력 :  752
팀2 전투력 :  820
팀2 승리


### 잉글랜드 VS 포르투갈은 포르투갈의 승리 예측
<hr>

In [24]:
# 로우 인덱스 팀이 컬럼 인덱스 팀과 경기했을 때 결과

result = DataFrame(data = [
                ['.', '승', '승', '승', '승'],
                ['패', '.', '패', '패', '패'],
                ['패', '승', '.', '패', '패'],
                ['패', '승', '승', '.', '승'],
                ['패', '승', '승', '패', '.'],
                ],
            index =  ['독일', '스페인', '잉글랜드', '포르투갈', '프랑스'],
            columns = ['독일', '스페인', '잉글랜드', '포르투갈', '프랑스'])
result

,독일,스페인,잉글랜드,포르투갈,프랑스
독일,.,승,승,승,승
스페인,패,.,패,패,패
잉글랜드,패,승,.,패,패
포르투갈,패,승,승,.,승
프랑스,패,승,승,패,.


## 8-2. 카타르 월드컵에서 실제로 경기했던 프랑스 VS 잉글랜드 점수 예측
- 카타르 월드컵의 8강전에 있던 프랑스 VS 잉글랜드의 점수를 타임라인을 보고 예측

In [25]:
# 우선 직전의 분석에서 프랑스가 이기는 것으로 예측
# 프랑스의 득점강세시간이 후반이며, 잉글랜드의 실점취약시간이 후반이므로 후반에 골을 넣을 것이라 추측

In [26]:
# 시간별 득점 타임라인

goaltime = timeline_data.loc[['프랑스', '잉글랜드'], '득점']
goaltime

,0분대,10분대,20분대,30분대,40분대,50분대,60분대,70분대,80분대,90분대,100분대,110분대,120분대
프랑스,1,3,2,2,3,9,4,3,3,0,0,0,0
잉글랜드,4,8,4,6,4,6,5,7,8,0,1,0,0


In [27]:
# 시간별 실점 타임라인

concededtime = timeline_data.loc[['프랑스', '잉글랜드'], '실점']
concededtime

,0분대,10분대,20분대,30분대,40분대,50분대,60분대,70분대,80분대,90분대,100분대,110분대,120분대
프랑스,1,1,0,5,2,1,2,0,3,1,0,0,0
잉글랜드,0,1,0,1,0,3,4,1,3,1,0,0,0


In [28]:
# 프랑스가 골을 넣을거라 예측되는 시간대 탐색

(goaltime.loc['프랑스'] + concededtime.loc['잉글랜드']).sort_values(ascending = False).head(5)

50분대    12
60분대     8
80분대     6
10분대     4
70분대     4
dtype: int64

In [29]:
# 잉글랜드가 골을 넣을거라 예측되는 시간대 탐색

(goaltime.loc['잉글랜드'] + concededtime.loc['프랑스']).sort_values(ascending = False).head(5)

30분대    11
80분대    11
10분대     9
50분대     7
60분대     7
dtype: int64

## 결과 예측

- 프랑스가 10 / 50 / 60 / 70 / 80 분대중 한 시간대에 두 골 이상을 넣을 것으로 추정
- 잉글랜드가 10 / 30 / 50 / 60 / 80 분대중 한 시간대에 한 골 이상을 넣을 것으로 추정
- 저 중 프랑스가 후반 득점이 있을 확률이 매우 크다!

<hr>

## 실제 결과
<img src="data/프랑스잉글랜드.png" alt="datetime" style="width: 800px;"/>

- 진짜 프랑스가 후반 득점에 골을 넣어 이겼다...!
- 심지어 3개의 득점 모두 예측했던 5개의 득점시간대 안에 있었다!
- 그러나 프랑스는 50분대에 골을 넣을 것이 유력했고, 잉글랜드는 30분 혹은 80분대에 골을 넣을 것이 유력했으나 이 점은 빗나갔다.
- 빅데이터... 무섭다!